<a href="https://colab.research.google.com/github/mavenzer/Autism-Detection-Using_YOLO/blob/master/Tutorial_implementing_Xception_in_TensorFlow_2_0_using_the_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tf-nightly-gpu-2.0-preview

In [0]:
import tensorflow as tf
print(tf.__version__)
tf.keras.backend.clear_session()

2.0.0-dev20190301


# Tutorial: implementing Xception in TensorFlow 2.0 using the Functional API

In this tutorial, we'll implement the Xception architecture for image classification, and we'll train it on a well-known Cats vs. Dogs image classification dataset from Kaggle.

## A step-by-step implementation of the original Xception model

Xception is a mid-2016 architecture ([published in CVPR 2017](https://arxiv.org/abs/1610.02357)) that heavily relies on Depthwise Separable convolutions. It's commonly used in the computer vision community for image segmentation tasks.

Here's what it looks like (image taken from the original paper):

![xception_flow](https://s3-us-west-1.amazonaws.com/fchollet-public-colabs/xception_flow.png)

The way we will implement Xception will strictly mirror the graph above. We will structure our code into 3 functions:

- `entry_flow`
- `middle_flow`
- `exit_flow`

Then we will chain these 3 functions to create our model.

Like this:

```python
def entry_flow(inputs):
  ...
  return x

def middle_flow(x, num_blocks=8):
  ...
  return x

def exit_flow(x, num_classes=1000):
  ...
  return preds

inputs = keras.Input(shape=(299, 299, 3))
outputs = exit_flow(middle_flow(entry_flow(inputs)))
model = keras.Model(inputs, outputs)
```

This means that we will be using the Functional API. We'll assume that you are already familiar with the "[Guide to the Keras Functional API in TensorFlow 2.0](https://colab.research.google.com/drive/1utjoLkLe3NamEQPw3n4E7kIV5qWjlHPa)".

Let's get started with the entry flow.

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

In [0]:

def entry_flow(inputs):
  # Entry block
  x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(64, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)

  previous_block_activation = x  # Set aside residual
  
  # Blocks 1, 2, 3 are identical apart from the feature depth.
  for size in [128, 256, 728]:
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(size, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(size, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Project residual
    residual = layers.Conv2D(
        size, 1, strides=2, padding='same')(previous_block_activation)
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

  return x


For debugging purposes, let's print the summary of the flow so far.
We won't actually use the `intermediate_model` object we create here, we just need it to call `.summary()` on it.

This helps us see how the inputs get downsampled (from `(299, 299, 3)` to `(19, 19, 728)`, as expected from the original model diagram) as they move down the flow.

In [0]:
inputs = keras.Input(shape=(299, 299, 3))
intermediate_outputs = entry_flow(inputs)
intermediate_model = keras.Model(inputs, intermediate_outputs)
intermediate_model.summary(line_length=120)

Model: "model"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   [(None, 299, 299, 3)]      0                                                     
________________________________________________________________________________________________________________________
conv2d (Conv2D)                        (None, 150, 150, 32)       896           input_1[0][0]                           
________________________________________________________________________________________________________________________
batch_normalization_v2 (BatchNormaliza (None, 150, 150, 32)       128           conv2d[0][0]                            
________________________________________________________________________________________________________________________
activation (Activ

Now let's move on to the middle flow:

In [0]:
def middle_flow(x, num_blocks=8):
  
  previous_block_activation = x

  for _ in range(num_blocks):
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, previous_block_activation])  # Add back residual
    previous_block_activation = x  # Set aside next residual
    
  return x

As with the entry flow, let's print the summary of what the flow so far looks like:

In [0]:
inputs = keras.Input(shape=(19, 19, 728))
intermediate_outputs = middle_flow(inputs)
intermediate_model = keras.Model(inputs, intermediate_outputs)
intermediate_model.summary(line_length=120)


Model: "model_1"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_2 (InputLayer)                   [(None, 19, 19, 728)]      0                                                     
________________________________________________________________________________________________________________________
activation_8 (Activation)              (None, 19, 19, 728)        0             input_2[0][0]                           
________________________________________________________________________________________________________________________
separable_conv2d_6 (SeparableConv2D)   (None, 19, 19, 728)        537264        activation_8[0][0]                      
________________________________________________________________________________________________________________________
batch_normaliza

We finish with the exit flow:

In [0]:
def exit_flow(x, num_classes=1000):

  previous_block_activation = x

  x = layers.Activation('relu')(x)
  x = layers.SeparableConv2D(728, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.Activation('relu')(x)
  x = layers.SeparableConv2D(1024, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  
  x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

  # Project residual
  residual = layers.Conv2D(
      1024, 1, strides=2, padding='same')(previous_block_activation)
  x = layers.add([x, residual])  # Add back residual
  
  x = layers.SeparableConv2D(1536, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  
  x = layers.SeparableConv2D(2048, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  
  x = layers.GlobalAveragePooling2D()(x)
  if num_classes == 1:
    activation = 'sigmoid'
  else:
    activation = 'softmax'
  return layers.Dense(num_classes, activation=activation)(x)

In [0]:
# Free up the previously created models (since the notebook is stateful)
keras.backend.clear_session()

inputs = keras.Input(shape=(19, 19, 728))
outputs = exit_flow(inputs)
intermediate_model = keras.Model(inputs, outputs)
intermediate_model.summary(line_length=120)
  

To recap, this is the entirety of our code so far:

In [0]:
def entry_flow(inputs):

  x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(64, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)

  previous_block_activation = x  # Set aside residual
  
  # Blocks 1, 2, 3 are identical apart from the feature depth.
  for size in [128, 256, 728]:
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(size, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(size, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    residual = layers.Conv2D(  # Project residual
        size, 1, strides=2, padding='same')(previous_block_activation)           
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

  return x


def middle_flow(x, num_blocks=8):
  
  previous_block_activation = x

  for _ in range(num_blocks):
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, previous_block_activation])  # Add back residual
    previous_block_activation = x  # Set aside next residual
    
  return x


def exit_flow(x, num_classes=1000):
  
  previous_block_activation = x

  x = layers.Activation('relu')(x)
  x = layers.SeparableConv2D(728, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.Activation('relu')(x)
  x = layers.SeparableConv2D(1024, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  
  x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

  residual = layers.Conv2D(  # Project residual
      1024, 1, strides=2, padding='same')(previous_block_activation)
  x = layers.add([x, residual])  # Add back residual
  
  x = layers.SeparableConv2D(1536, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  
  x = layers.SeparableConv2D(2048, 3, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  
  x = layers.GlobalAveragePooling2D()(x)
  if num_classes == 1:
    activation = 'sigmoid'
  else:
    activation = 'softmax'
  return layers.Dense(num_classes, activation=activation)(x)


# Free up the previously created models (since the notebook is stateful)
keras.backend.clear_session()

# Create Xception by chaining the 3 flows
inputs = keras.Input(shape=(299, 299, 3))
outputs = exit_flow(middle_flow(entry_flow(inputs)))
xception = keras.Model(inputs, outputs)

Notice how there is a one-to-one correspondence between the model diagram we were working from, and the code we just wrote.

Note that it is also possible to build the model on top of variable-size images, so that you can process images of different sizes, or images of a size that is unknown in advance:

In [0]:
# Free up the previously created models (since the notebook is stateful)
keras.backend.clear_session()

inputs = keras.Input(shape=(None, None, 3))  # Variable-size image inputs.
outputs = exit_flow(middle_flow(entry_flow(inputs)))
xception = keras.Model(inputs, outputs)